This is a `bqplot` recreation of Mike Bostock's [Wealth of Nations](https://bost.ocks.org/mike/nations/). This was also done by [Gapminder](http://www.gapminder.org/world/#$majorMode=chart$is;shi=t;ly=2003;lb=f;il=t;fs=11;al=30;stl=t;st=t;nsl=t;se=t$wst;tts=C$ts;sp=5.59290322580644;ti=2013$zpv;v=0$inc_x;mmid=XCOORDS;iid=phAwcNAVuyj1jiMAkmq1iMg;by=ind$inc_y;mmid=YCOORDS;iid=phAwcNAVuyj2tPLxKvvnNPA;by=ind$inc_s;uniValue=8.21;iid=phAwcNAVuyj0XOoBL_n5tAQ;by=ind$inc_c;uniValue=255;gid=CATID0;by=grp$map_x;scale=log;dataMin=194;dataMax=96846$map_y;scale=lin;dataMin=23;dataMax=86$map_s;sma=49;smi=2.65$cd;bd=0$inds=;modified=60). It is originally based on a TED Talk by [Hans Rosling](http://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen).

#### Required imports

In [ ]:
import pandas as pd
from bqplot import (LogScale, LinearScale, OrdinalColorScale, ColorAxis, Axis, Scatter, CATEGORY10,
                    Label, Figure)
from bqplot.default_tooltip import Tooltip
from ipywidgets import VBox, IntSlider, Button
from IPython.display import display
import os
import numpy as np
from time import sleep

### Cleaning and Formatting JSON Data

In [ ]:
data = pd.read_json(os.path.abspath('data_files/nations.json'))

In [ ]:
def clean_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data

def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range

def extrap_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data[column] = data[column].apply(extrap_interp)
    return data

In [ ]:
data = clean_data(data)
data = extrap_data(data)

In [ ]:
income_min, income_max = np.min(data['income'].apply(np.min)), np.max(data['income'].apply(np.max))
life_exp_min, life_exp_max = np.min(data['lifeExpectancy'].apply(np.min)), np.max(data['lifeExpectancy'].apply(np.max))
pop_min, pop_max = np.min(data['population'].apply(np.min)), np.max(data['population'].apply(np.max))

#### Helper functions for parsing the data

In [ ]:
def get_data(year_index):
    income = data['income'].apply(lambda x: x[year_index])
    life_exp = data['lifeExpectancy'].apply(lambda x: x[year_index])
    pop =  data['population'].apply(lambda x: x[year_index])
    return income, life_exp, pop

### Defining Scales

In [ ]:
x_sc = LogScale(min=income_min, max=income_max)
y_sc = LinearScale(min=life_exp_min, max=life_exp_max)
c_sc = OrdinalColorScale(domain=data['region'].unique().tolist(), colors=CATEGORY10[:6])
size_sc = LinearScale(min=pop_min, max=pop_max)

### Defining the Axes

In [ ]:
ax_y = Axis(label='Life Expectancy', scale=y_sc, orientation='vertical', side='left')
ax_x = Axis(label='Income per Capita', scale=x_sc)

### Creating a Slider for the years

In [ ]:
year_slider = IntSlider(min=1800, max=2008, step=1, description='Year', value=1800)

### Creating a Button for the animation

In [ ]:
animate_button = Button(description='Play', background_color='MediumSeaGreen', color='Black', icon='fa-play')

### Creating the Tooltip to display the required fields

In [ ]:
tt = Tooltip(fields=['name', 'x', 'y'], labels=['Country Name', 'Income per Capita', 'Life Expectancy'])

### Creating the Label to display the year

In [ ]:
year_label = Label(x=0.85, y=0.1, font_size='52px', font_weight='bolder', color='orange', text=str(year_slider.value))

### Creating the Scatter Mark with the appropriate size and color parameters passed

In [ ]:
cap_income, life_exp, pop = get_data(year_slider.value-1800)

In [ ]:
wealth_scat = Scatter(x=cap_income, y=life_exp, color=data['region'], size=pop,
                      names=data['name'], display_names=False,
                      scales={'x': x_sc, 'y': y_sc, 'color': c_sc, 'size': size_sc},
                      default_size=4112, tooltip=tt, animate=True, stroke='Black')

### Creating the Figure

In [ ]:
fig = Figure(marks=[wealth_scat, year_label], axes=[ax_x, ax_y],
             title='Health and Wealth of Nations', fig_color='White',
             animation_duration=100)

### Defining the callback for the slider

In [ ]:
def year_changed(new):
    wealth_scat.x, wealth_scat.y, wealth_scat.size = get_data(year_slider.value-1800)
    year_label.text = str(year_slider.value)
    
year_slider.observe(year_changed, 'value')

### Defining the callback for the button

In [ ]:
def button_clicked(value):
    animate_button.visible = False
    for i in range(1800, 2009, 1):
        year_slider.value = i
        sleep(0.05)
    animate_button.visible = True

animate_button.on_click(button_clicked)

### Displaying the GUI

In [ ]:
display(VBox([animate_button, fig, year_slider]))